In [32]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import cv2

In [33]:
import os
import pandas as pd
#from scipy.misc import imread
from skimage.io import imread
import scipy.misc
import math
import numpy as np
import cv2
import keras
import seaborn as sns
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential

# from the follwing https://towardsdatascience.com/traffic-sign-detection-using-convolutional-neural-network-660fb32fe90e 
# trying to understand the basics of using keras and the paramters vs the hyperparamters

In [34]:
# %pylab inline
# import os
# import numpy as np
# import pandas as pd
# import scipy.misc
# #from scipy.misc import imread
# from sklearn.metrics import accuracy_score
# import tensorflow as tf
# import keras
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
# from keras.optimizers import Adam

In [35]:
#this loaded csv files
data_dir = os.path.realpath('./Images2/')

train = pd.read_csv(os.path.join(data_dir, 'train', 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test', 'test.csv'))

#sample_submission = pd.read_csv(os.path.join(data_dir, 'Sample_Submission.csv'))


In [36]:
train

,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3
...,...,...
48995,48995.png,2
48996,48996.png,4
48997,48997.png,9
48998,48998.png,3


In [37]:
temp = []
for img_name in train.filename:
    image_path = os.path.join(data_dir, 'train', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
train_x = np.stack(temp)
print(train_x.shape)
train_x /= 255.0
train_x = train_x.reshape(-1, 784).astype('float32')
print(train_x.shape)


(49000, 28, 28)
(49000, 784)


In [38]:
temp = []
for img_name in test.filename:
    image_path = os.path.join(data_dir, 'test', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
test_x = np.stack(temp)

test_x /= 255.0
test_x = test_x.reshape(-1, 784).astype('float32')


In [39]:
print(test_x.shape)

(21000, 784)


In [40]:
train_y = keras.utils.np_utils.to_categorical(train.label.values)

split_size = int(train_x.shape[0]*0.7)

In [41]:
# split training dataset into train and validation dataset
train_x, val_x = train_x[:24500], train_x[24500:]
train_y, val_y = train_y[:24500], train_y[24500:]

In [42]:
# reshape data as the keras model requires

train_x_temp = train_x.reshape(-1, 28, 28, 1)
val_x_temp = val_x.reshape(-1, 28, 28, 1)

In [43]:
# define vars
input_shape = (784,)
input_reshape = (28, 28, 1)

hidden_num_units = 2048
hidden_num_units1 = 1024
hidden_num_units2 = 128
output_num_units = 10

epochs = 2 #chagned the epoch time to save time.
batch_size = 16

In [44]:
# # Max-pool reduces the size of inputs, by taking the largest pixel-value from a grid
# nn.add(MaxPooling2D(pool_size=(2, 2)))
# nn.add(Conv2D(64, 3, 3, activation='relu'))
# nn.add(MaxPooling2D(pool_size=(2, 2)))
# nn.add(Flatten())
# nn.add(Dense(128, activation='relu'))
# nn.add(Dense(10, activation='softmax'))

In [45]:
#creating a model with different paramters that can be tunned to see which will prform the best
model = Sequential([

 Conv2D(16, (3, 3), activation='relu', input_shape=input_reshape, padding='same'),
 BatchNormalization(),

 Conv2D(16, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
 MaxPooling2D(pool_size=(2, 2)),
 Dropout(0.2),
    
 Conv2D(32, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
    
 Conv2D(32, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
 MaxPooling2D(pool_size=(2, 2)),
 Dropout(0.2),
    
 Conv2D(64, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
    
 Conv2D(64, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
 MaxPooling2D(pool_size=(2, 2)),
 Dropout(0.2),

 Flatten(),

 Dense(units=hidden_num_units, activation='relu'),
 Dropout(0.3),
 Dense(units=hidden_num_units1, activation='relu'),
 Dropout(0.3),
 Dense(units=hidden_num_units2, activation='relu'),
 Dropout(0.3),
 Dense(units=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

In [46]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

In [47]:
trained_model_conv = model.fit(train_x_temp, train_y, epochs=epochs, batch_size=batch_size, 
                               validation_data=(val_x_temp, val_y))

Train on 24500 samples, validate on 24500 samples
Epoch 1/2
24500/24500 [==============================] - 423s 17ms/step - loss: 0.7010 - accuracy: 0.7685 - val_loss: 0.2433 - val_accuracy: 0.9227
Epoch 2/2
24500/24500 [==============================] - 416s 17ms/step - loss: 0.2020 - accuracy: 0.9380 - val_loss: 0.1396 - val_accuracy: 0.9559


In [48]:
test_x_temp = test_x.reshape(-1, 28, 28, 1)
pred = model.predict_classes(test_x_temp)